<a href="https://colab.research.google.com/github/valentinocc/Keras_cifar10/blob/master/cifar10_cnn_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed May 29 10:42:05 2019

@author: val

basic CNN for cifar10 dataset
"""
import numpy as np
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split


In [0]:
def model(input_shape):

    model = tf.keras.models.Sequential()
    convolution_batchnorm_relu_block(model, 32, (3, 3), 'same', input_shape = input_shape, first_layer = True)
    convolution_batchnorm_relu_block(model, 32, (3, 3), 'same')
    model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
    
    convolution_batchnorm_relu_block(model, 64, (3, 3), 'same')
    convolution_batchnorm_relu_block(model, 64, (3, 3), 'same')
    model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
    
    model.add(tf.keras.layers.Flatten())
    dense_batchnorm_relu_block(model, 256)
    model.add(tf.keras.layers.Dense(10))
    model.add(tf.keras.layers.Activation('softmax'))
    
    model.summary()
    
    return model

def convolution_batchnorm_relu_block(model, filters, filter_shape, padding_setting, input_shape = None, first_layer = False):

    if (first_layer):
        model.add(tf.keras.layers.Conv2D(filters, filter_shape, padding = padding_setting, data_format = 'channels_last', input_shape = input_shape))
    else:
        model.add(tf.keras.layers.Conv2D(filters, filter_shape, padding = padding_setting))
    
    model.add(tf.keras.layers.BatchNormalization(axis = 3))
    model.add(tf.keras.layers.Activation('relu'))

def dense_batchnorm_relu_block(model, units):
    
    model.add(tf.keras.layers.Dense(units))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.006*8, shuffle= True)


In [15]:
input_shape = X_train.shape[1:]
keras_model = model(input_shape)

TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tpu_model = tf.contrib.tpu.keras_to_tpu_model(keras_model, strategy = tf.contrib.tpu.TPUDistributionStrategy(tf.contrib.cluster_resolver.TPUClusterResolver(tpu = TPU_WORKER)))
tpu_model.compile( optimizer = tf.train.AdamOptimizer(learning_rate = 1e-3), loss = tf.keras.losses.sparse_categorical_crossentropy, metrics = ['sparse_categorical_accuracy'])
model_convergence_data = tpu_model.fit(x_train, y_train, epochs = 20, batch_size = 32*8, validation_data = (x_valid, y_valid))
tpu_model.evaluate(X_test, y_test, batch_size = 32*8)
tpu_model.save_weights('./tpu_model_weights.h5', overwrite = True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_v1_15 (B (None, 32, 32, 32)        128       
_________________________________________________________________
activation_18 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_v1_16 (B (None, 32, 32, 32)        128       
_________________________________________________________________
activation_19 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 32)        0         
__________